In [1]:
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/auxiliary_files/'

1) convert sql dump into csv file  
see https://stackoverflow.com/questions/27584405/how-to-import-a-mysqldump-into-pandas

In [2]:
from io import StringIO
import re, shutil
import os.path

In [3]:
import pandas as pd

In [4]:
import csv

In [5]:
reg1 = re.compile('`([\w_]+)`')
reg2 = re.compile(r'\((?P<val>\d+?,\'[a-z]*?\',\'.*?(?:(?:\\\\)|(?<!\\))\')\)')

FN_UK_LANGLINKS = 'ukwiki-20180920-langlinks.sql'
dump_filename = PATH_TO_DATA+FN_UK_LANGLINKS
target_table = "langlinks"

sio = StringIO()

read_mode = 0 # 0 - skip, 1 - header, 2 - data
with open(dump_filename, 'r', encoding='ISO-8859-1') as f:
    for line in f:
        line = line.strip()
        if line.lower().startswith('insert') and target_table in line:
            read_mode = 2
        if line.lower().startswith('create table') and target_table in line:
            read_mode = 1
            continue

        if read_mode==0:
            continue

        # Filling up the headers
        elif read_mode==1:
            if line.lower().startswith('primary'):
                # add more conditions here for different cases 
                #(e.g. when simply a key is defined, or no key is defined)
                read_mode=0
                sio.seek(sio.tell()-1) # delete last comma
                sio.write('\n')
                continue
            colheader = re.findall(reg1,line)
            for col in colheader:
                sio.write(col.strip())
                sio.write(',')

        # Filling up the data -same as @firelynx's code
        elif read_mode ==2:
            for match in reg2.finditer(line): 
                newline = match.group("val")
                #print(newline)
                newline = newline.strip(' ()')
                #print(newline)
                newline = newline.replace('`', '')
                #print(newline)
                sio.write(newline)
                sio.write("\n")
            #if line.endswith(';'):
                #print("END OF INSERT")
sio.seek(0)
with open (PATH_TO_DATA+'ukwiki-20180920-langlinks.csv', 'w', encoding='ISO-8859-1') as fd:
    shutil.copyfileobj(sio, fd,-1)

In [6]:
df_uk_langlinks = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-langlinks.csv', encoding='ISO-8859-1', quotechar="'", escapechar ="\\")
df_uk_langlinks

,ll_from,ll_lang,ll_title
0,25610,aa,Category:User de
1,25692,aa,Category:User de-1
2,159531,aa,Template:Babel
3,269084,aa,Template:Cc-by-sa-2.0
4,35964,aa,Template:Delete
5,1987209,aa,Template:FlowMention
6,244518,aa,Template:Style/IPA
7,244517,aa,Template:Style/yid
8,25620,aa,Template:User en-3
9,41011,aa,Template:User es-2


In [7]:
df_uk_en_langlinks = df_uk_langlinks[df_uk_langlinks.ll_lang == 'en']
df_uk_en_langlinks.shape

(599636, 3)

In [8]:
df_uk_en_langlinks.to_csv(PATH_TO_DATA+'ukwiki-20180920-langlinks_en.csv', index = False, encoding='ISO-8859-1', quotechar="'", escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)

In [9]:
df_uk_en_langlinks.to_csv(PATH_TO_DATA+'ukwiki-20180920-langlinks_en.csv', 
                     index = False, encoding='ISO-8859-1', quotechar="\"", sep ="^")

In [10]:
df_en_langlinks = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-langlinks_en.csv', encoding='ISO-8859-1', sep="^")

In [11]:
df_en_langlinks

,ll_from,ll_lang,ll_title
0,3,en,NaN
1,93215,en,NaN
2,109484,en,NaN
3,122051,en,NaN
4,130506,en,NaN
5,609236,en,NaN
6,621969,en,NaN
7,772924,en,NaN
8,907757,en,NaN
9,942309,en,NaN


In [12]:
df_en_langlinks_en_title = df_en_langlinks[['ll_title']]

In [19]:
df_en_langlinks_en_title.to_dict()

{'ll_title': {0: nan,
  1: nan,
  2: nan,
  3: nan,
  4: nan,
  5: nan,
  6: nan,
  7: nan,
  8: nan,
  9: nan,
  10: nan,
  11: nan,
  12: nan,
  13: nan,
  14: nan,
  15: nan,
  16: nan,
  17: nan,
  18: nan,
  19: nan,
  20: nan,
  21: nan,
  22: nan,
  23: '! (The Dismemberment Plan album)',
  24: '! (disambiguation)',
  25: '!!',
  26: '!!!',
  27: '!Action Pact!',
  28: '!T.O.O.H.!',
  29: '"Hello, World!" program',
  30: '"Heroes" (David Bowie album)',
  31: '"Heroes" (David Bowie song)',
  32: '"No Place to Hide" (Lost in Space)',
  33: '"Repent, Harlequin!" Said the Ticktockman',
  34: '"Weird Al" Yankovic',
  35: '"â\x80\x94And He Built a Crooked Houseâ\x80\x94"',
  36: '$ (disambiguation)',
  37: '$+',
  38: '$+',
  39: '$+',
  40: '$+',
  41: '$+',
  42: '$+',
  43: '$+',
  44: "','",
  45: "','",
  46: "'03 Bonnie & Clyde",
  47: "'39",
  48: "'50s on 5",
  49: "'60s on 6",
  50: "'74 Jailbreak",
  51: "'92 Tour EP",
  52: "'98 Live Meltdown",
  53: "'Adud al-Dawla",
  54:

In [21]:
f = lambda x: str(x).encode('ISO-8859-1')
df_en_langlinks_en_title = df_en_langlinks_en_title.applymap(f)

In [22]:
f_decode = lambda x: x.decode('utf8')
df_en_langlinks_en_title = df_en_langlinks_en_title.applymap(f_decode)

In [23]:
df_en_langlinks_en_title

,ll_title
0,nan
1,nan
2,nan
3,nan
4,nan
5,nan
6,nan
7,nan
8,nan
9,nan


In [26]:
df_en_langlinks_en_title.columns = ['ll_title_decoded']

In [28]:
df_en_langlinks_decoded = pd.concat([df_en_langlinks, df_en_langlinks_en_title], axis=1)

In [29]:
df_en_langlinks_decoded = df_en_langlinks_decoded[['ll_from',]]

,ll_from,ll_lang,ll_title,ll_title_decoded
0,3,en,NaN,nan
1,93215,en,NaN,nan
2,109484,en,NaN,nan
3,122051,en,NaN,nan
4,130506,en,NaN,nan
5,609236,en,NaN,nan
6,621969,en,NaN,nan
7,772924,en,NaN,nan
8,907757,en,NaN,nan
9,942309,en,NaN,nan


In [ ]:
df_uk_en_langlinks.to_csv(PATH_TO_DATA+'ukwiki-20180920-langlinks_en.csv', 
                     index = False, encoding='ISO-8859-1', quotechar="\"", sep ="^")